In [ ]:
import pandas as pd
import os
import datetime

In [ ]:
def sort_categories(security):
    if 'Call' in security:
        return 'call'
    elif 'Put' in security:
        return 'put'
    else:
        return 'stock'
    
def get_symbol(security, category):
    if category=='stock':
        return security.split()[-1][1:-1]
    else:
        return security
    
def get_underlying(symbol):
    try:
        underlying = symbol.split()[0]
        return underlying
    except BaseException:
        return
    
def get_hold_duration(open_date, close_date):
    t0 = datetime.datetime.strptime(open_date, '%Y-%m-%d %H:%M:%S')
    t1 = datetime.datetime.strptime(close_date, '%Y-%m-%d %H:%M:%S')
    return (t1-t0)

In [ ]:
data_path = './data'
files = ['RC_869434357_20150101_20151231.xlsx', 'RC_869434357_20160101_20161231.xlsx', 'RC_869434357_20170101_20171231.xlsx',
         'RC_869434357_20180101_20181231.xlsx','RC_869434357_20190101_20191231.xlsx', 'RC_869434357_20200101_20201231.xlsx', 
         'RC_869434357_20210101_20210501.xlsx']   

In [ ]:
data = []
for i in files[:3]:
    file_path = os.path.join(data_path, i)
    data.append(pd.read_excel(file_path, dtype='str')[:-1])

In [ ]:
df = pd.concat(data)

In [ ]:
df.columns = [i.lower().replace(' ', '_') for i in df.columns]
# df = df[df['trans_type'] != 'Wash Sale Adj']
# df = df[df['trans_type'] != 'Short WS Adj']
df.head()

In [ ]:
df.shape

In [ ]:
df['category'] = df['security'].apply(lambda x: sort_categories(x))
df['symbol'] = df.apply(lambda x: get_symbol(x.security, x.category), axis=1)
df['underlying'] = df['symbol'].apply(lambda x: get_underlying(x))
df['hold_duration'] = df.apply(lambda x: get_hold_duration(x.open_date, x.close_date), axis=1)
df['hold_duration'] = df.apply(lambda x: datetime.timedelta(0) if x.trans_type=='Wash Sale Adj' else x.hold_duration, axis=1)
df['hold_duration'] = df.apply(lambda x: datetime.timedelta(0) if x.trans_type=='Short WS Adj' else x.hold_duration, axis=1)

In [ ]:
df.to_excel('./data/performance.xlsx', index=False)